# Fake News Classication

Read the two datasets:
1. ABC news article titles for 14 years
2. The Examiner news articles

## Data setup and cleaning

In [ ]:
import pandas

data_set_loc = '/Users/koo01a/Desktop/fake_news'

# Read in white wine data 
# https://www.kaggle.com/therohk/million-headlines
abc = pandas.DataFrame(pandas.read_csv(data_set_loc+"/abcnews-date-text.csv", sep=','))

# Read in red wine data 
# https://www.kaggle.com/therohk/examine-the-examiner
examiner = pandas.DataFrame(pandas.read_csv(data_set_loc+"/examiner-date-text.csv", sep=','))

print(abc.info())
print(examiner.info())

In [ ]:
print(abc.head())
from lolviz import *
g = listviz(['hi','mom',{3,4},{"parrt":"user"}])
g.render(view=True) # render graphviz.files.Source object

Assign examiner as fake (fake=1) and abs as not fake (fake=0)

In [ ]:
# Add `type` column to `red` with value 1
examiner['fake'] = 1

# Add `type` column to `white` with value 0
abc['fake'] = 0

# Append `white` to `red`
articles = pandas.concat([examiner,abc]).sort_index().reset_index(drop=True)[:100000]
#articles = examiner.append(abc, ignore_index=True)

In [ ]:
print(articles.head())

In [ ]:
articles['headline_text'] = [str(headline) for headline in articles['headline_text']]

In [ ]:
from keras.preprocessing.text import *

In [ ]:


MAX_WORDS=30000

tokenizer = Tokenizer()

tokenizer.fit_on_texts(articles['headline_text'])

X = tokenizer.texts_to_matrix(articles['headline_text'], mode='binary')
y = y=np.ravel(articles['fake'])

#tokenizer.texts_to_sequences(input)


In [ ]:
len(tokenizer.word_index)
tokenizer.texts_to_matrix(["to in the", "the brisbane winter","surf"])
y

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

vocab_size = len(tokenizer.word_index) + 1

model = Sequential()

model.add(Dense(500, activation="relu", input_dim=len(tokenizer.word_index) + 1))
model.add(Dense(80, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5, validation_split=.5, verbose=1)

In [ ]:
model.save('fake-news-classifier.h5')

In [ ]:
predictions = model.predict_classes(X)

## Try some classification of arbitrary text 

In [ ]:
input_text = "10 best holiday destinations."
input_vector = tokenizer.texts_to_matrix([input_text], mode='binary')
prob_fake = model.predict(input_vector)[0][0]

print("Classifier thinks this is%s fake (p=%s)." % (" not" if prob_fake < 0.5 else "", round(prob_fake,4)))

## Incorrect classifications

In [ ]:
articles['predictions'] = predictions
errors = [(articles['headline_text'][i], articles['fake'][i], articles['predictions'][i]) for i in range(0,len(articles)) if articles['fake'][i] != articles['predictions'][i]]


## Evaluate

In [ ]:
score = model.evaluate(X, y,verbose=1)

print(score)

In [ ]:
precision_score(y, predictions)

In [ ]:
recall_score(y, predictions)

In [ ]:
f1_score(y, predictions)